# Generate figures for publication
_[work in progress]_

##  STA analysis

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from neuroprocessing.imagingtrials import ImagingTrialLoader
from skimage.filters import gaussian


def plot_STA_img_and_dff(imaging_trial, trial_ROI, bleach_subtract = True):
    mask = imaging_trial.load_mask()
    _, t, dff = imaging_trial.get_sta_stack(1, 5, .5, trial_ROI)  # ssc_roi
    sta_df, _, _ = imaging_trial.get_sta_stack(1, 5, .5)  # ssc_roi

    # remove linear trend from dff (remnants of bleaching)
    # only fit first and last 3 elements
    if bleach_subtract:
        dff = dff - np.polyval(np.polyfit(np.array([t[:3], t[-3:]]).flatten(),
                                        np.array([dff[:3], dff[-3:]]).flatten(), 1),
                                        t)

    # set everything outside the mask to 0
    sta_df = sta_df * mask
    sta_df = sta_df.mean(axis=0)
    # add xy blur to sta_df image

    sta_df = gaussian(sta_df, sigma=2, channel_axis=0)
    sta_df_max = sta_df.max(axis=0)

    f, axs = plt.subplots(1, 2, figsize=(10, 5))
    cbar = plt.colorbar(axs[0].imshow(sta_df_max, cmap='inferno'), ax=axs[0])
    axs[0].imshow(sta_df_max, cmap='inferno', aspect='auto')
    axs[0].axis('off')
    axs[1].plot(t, dff)
    axs[1].set_xlabel('Time (s)')
    axs[1].set_ylabel('dF/F')

    axs[0].invert_xaxis()
    f.suptitle(imaging_trial)

    plt.tight_layout()

ssc_LHL_roi = {
    "center": (125,175),  # (x, y)
    "width": 80,
    "height": 80
}
ssc_RHL_roi = {
    "center": (265,175),  # (x, y)
    "width": 80,
    "height": 80
}

# RHL trial
imaging_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
imaging_trials.filter(exp_dir='2024-02-29',# "2024-03-19",
limb='RHL',
rec_time='5min',
# remainder='.*pt25pctISO_2'
)
trial = imaging_trials.trials[0]

plot_STA_img_and_dff(trial, ssc_RHL_roi)



In [ ]:

# LHL trial
imaging_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
imaging_trials.filter(exp_dir='2024-02-29',# "2024-03-19",
limb='LHL',
rec_time='5min',
remainder='.*pt25pctISO_2'
)
trial = imaging_trials.trials[0]

plot_STA_img_and_dff(trial, ssc_LHL_roi)

In [ ]:
ssc_RFL_roi = {
    "center": (335,210),  # (x, y)
    "width": 50,
    "height": 50
}
# RFL trial
imaging_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
imaging_trials.filter(exp_dir='2024-02-21',# "2024-03-19",
limb='RFL',
rec_time='5min',
# remainder='.*pt25pctISO_2'
)
trial = imaging_trials.trials[0]

plot_STA_img_and_dff(trial, ssc_RFL_roi)

# Injection analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuroprocessing.imagingtrials import ImagingTrialLoader

params = {
        "downsample_factor": 8,
        "aligner_target_num_features": 700,
        "secs_before_stim": 60, # only process frames starting at X seconds before stimulus
        "preprocess_prefix": "aligned_downsampled_",
        "process_prefix": 'processed_',
        "s3fs_toplvl_path": "/Users/ilya_arcadia/arcadia-neuroimaging-pruritogens/Videos",
        "local_toplvl_path": "/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections/",
        "load_from_s3": False,
        "save_to_s3": False,
        'crop_px' : 20,
        'bottom_percentile' : 5
        }

trials = ImagingTrialLoader(params)
exp_dates, trial_names = trials.filter_exp_and_trial_dirs(exp_dir="2024-03-20-F1Num8REP",
                                                          limb='RHL'
                                                        )
# exp_dates, trial_names = trials.filter_exp_and_trial_dirs()

masks = trials.load_mask_files()
traces = trials.load_traces()
sync_infos = trials.get_sync_infos()
# Plot the traces

fig, ax = plt.subplots(figsize=(5,3))
for trace, exp_date, trial_name, sync in zip(traces, exp_dates, trial_names, sync_infos, strict=True):
    t = (np.arange(0, len(trace))) / (sync['framerate_hz'] / params['downsample_factor']) - params['secs_before_stim']
    trace = trace - trace[np.where(t >= 0)[0][0]]
    # remove slope
    trace = trace - np.polyval(np.polyfit(t, trace, 1), t)
    ax.plot(t, trace, label=exp_date + ' ' + trial_name)
# legend to the right of the plot
# ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.axvline(x=0, color='r', linestyle='--')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Response (a.u.)')
ax.set_xlim([-50, 600])
# plot masks
fig, axs = plt.subplots(ncols=len(masks), figsize=(15, 7))
for ax, mask, exp_date, trial_name in zip(axs, masks, exp_dates, trial_names, strict=True):
    ax.imshow(mask, cmap='gray')
    # remove axis ticks
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(exp_date + ' ' + trial_name, fontsize=8)
plt.tight_layout()
